In [ ]:
!pip uninstall -y tf-nightly keras-nightly protobuf
!pip install tensorflow==2.20.0

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

train_df["label"] = train_df["label"].map({"ham": 0, "spam": 1})
test_df["label"] = test_df["label"].map({"ham": 0, "spam": 1})

In [ ]:
max_tokens = 5000
output_sequence_length = 100

vectorizer = layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length)
vectorizer.adapt(train_df["message"].values)

In [ ]:
model = keras.Sequential([
    vectorizer,
    layers.Embedding(input_dim=max_tokens, output_dim=16),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(
    train_df["message"].values,
    train_df["label"].values,
    epochs=20,
    validation_data=(test_df["message"].values, test_df["label"].values)
)

In [ ]:
def predict_message(pred_text):
  input_tensor = tf.convert_to_tensor([pred_text])
  prob = model.predict(input_tensor)[0][0]
  label = "spam" if prob > 0.5 else "ham"
  return [float(prob), label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
